
# Assignment 2 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 2, PROBLEM 1
Maximum Points = 8


A courier company operates a fleet of delivery trucks that make deliveries to different parts of the city. The trucks are equipped with GPS tracking devices that record the location of each truck at regular intervals. The locations are divided into three regions: downtown, the suburbs, and the countryside. The following table shows the probabilities of a truck transitioning between these regions at each time step:

| Current region | Probability of transitioning to downtown | Probability of transitioning to the suburbs | Probability of transitioning to the countryside |
|----------------|--------------------------------------------|-----------------------------------------------|------------------------------------------------|
| Downtown       | 0.3                                      | 0.4                                           | 0.3                                            |
| Suburbs        | 0.2                                      | 0.5                                           | 0.3                                            |
| Countryside    | 0.4                                      | 0.3                                           | 0.3                                            |

1. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region after two time steps? [1.5p]
2. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region **the first time** after two time steps? [1.5p]
3. Is this Markov chain irreducible? [1.5p]
4. What is the stationary distribution? [1.5p]
5. Advanced question: What is the expected number of steps until the first time one enters the downtown region having started in the suburbs region. Hint: to get within 1 decimal point, it is enough to compute the probabilities for hitting times below 30. [2p]



In [62]:
import numpy as np
P = np.matrix([[3/10,4/10,3/10],[1/5,1/2,3/10],[2/5,3/10,3/10]])
P            



matrix([[0.3, 0.4, 0.3],
        [0.2, 0.5, 0.3],
        [0.4, 0.3, 0.3]])

In [63]:
p0 = np.matrix((0,1,0))    # initial probability vector for a dry day zero
p0

matrix([[0, 1, 0]])

In [64]:
p2 = p0*P**2       # left multiply the initial probability vector by square of P
p2[0,0]


np.float64(0.28)

In [65]:
# Part 1

# Fill in the answer to part 1 below as a decimal number
problem1_p1 = 0.28

In [66]:
# Part 2

# Fill in the answer to part 2 below as a decimal number
problem1_p2 = 0.22

In [67]:
# Part 3

# Fill in the answer to part 3 below as a boolean
problem1_irreducible = True

In [68]:
# Part 4

A = P.T - np.eye(3)
A[-1] = np.ones(3)
b = np.zeros(3); b[-1] = 1

pi = np.linalg.solve(A, b)
#print(pi)

# Fill in the answer to part 4 below
# the answer should be a numpy array of length 3
# make sure that the entries sums to 1!
problem1_stationary = pi

In [69]:
SC = P[[1, 2]][:, [1, 2]]
SC_D = P[[1,2],0]


alpha = np.array([1.0, 0.0]) 
max_t = 30
pmf = []         
expect_approx = 0.0
alpha_k = alpha.copy()

for t in range(1, max_t + 1):
    # probability of first hit at time t: be still outside D for t-1 steps, then go to D
    p_hit_t = float(alpha_k @ SC_D)
    pmf.append(p_hit_t)
    expect_approx += t * p_hit_t
    # update mass that remains outside Downtown for next step
    alpha_k = alpha_k @ SC


print(f"Approx E[T_D | start=Suburbs] (sum to 30): {expect_approx:.2f}")




Approx E[T_D | start=Suburbs] (sum to 30): 3.84


/var/folders/tc/lq9907c548141k_92yrhy0r40000gn/T/ipykernel_54539/4165627324.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p_hit_t = float(alpha_k @ SC_D)


In [70]:
# Part 5

# Fill in the answer to part 5 below
# That is, the expected number of steps as a decimal number
problem1_ET = 3.846

---
## Assignment 2, PROBLEM 2
Maximum Points = 4


Use the **Multi-dimensional Constrained Optimisation** example (in `07-Optimization.ipynb`) to numerically find the MLe for the mean and variance parameter based on `normallySimulatedDataSamples`, an array obtained by a specific simulation of $30$ IID samples from the $Normal(10,2)$ random variable.

Recall that $Normal(\mu, \sigma^2)$ RV has the probability density function given by:

$$
f(x ;\mu, \sigma) = \displaystyle\frac{1}{\sigma\sqrt{2\pi}}\exp\left(\frac{-1}{2\sigma^2}(x-\mu)^2\right)
$$

The two parameters, $\mu \in \mathbb{R} := (-\infty,\infty)$ and $\sigma \in (0,\infty)$, are sometimes referred to as the location and scale parameters.

You know that the log likelihood function for $n$ IID samples from a Normal RV with parameters $\mu$ and $\sigma$ simply follows from $\sum_{i=1}^n \log(f(x_i; \mu,\sigma))$, based on the IID assumption. 

NOTE: When setting bounding boxes for $\mu$ and $\sigma$ try to start with some guesses like $[-20,20]$ and $[0.1,5.0]$ and make it larger if the solution is at the boundary. Making the left bounding-point for $\sigma$ too close to $0.0$ will cause division by zero Warnings. Other numerical instabilities can happen in such iterative numerical solutions to the MLe. You need to be patient and learn by trial-and-error. You will see the mathematical theory in more details in a future course in scientific computing/optimisation. So don't worry too much now except learning to use it for our problems.  

In [71]:

import numpy as np
from scipy import optimize
# do NOT change the next three lines
np.random.seed(123456) # set seed
# simulate 30 IID samples drawn from Normal(10,2)RV
normallySimulatedDataSamples = np.random.normal(10,2,30) 

# define the negative log likelihoo function you want to minimise by editing XXX
def negLogLklOfIIDNormalSamples(parameters):
    '''return the -log(likelihood) of normallySimulatedDataSamples with mean and var parameters'''
    mu_param=parameters[0]
    sigma_param=parameters[1]

    if sigma_param <= 0:     
        return np.inf
    x   = normallySimulatedDataSamples
    n   = x.size
    sse = np.sum((x - mu_param)**2)
   
    return n*np.log(sigma_param) + 0.5*n*np.log(2*np.pi) + sse/(2*sigma_param**2)

# sensible bounds and starting point
mu_init    = float(np.mean(normallySimulatedDataSamples))
sigma_init = float(np.std(normallySimulatedDataSamples, ddof=0))

# you should only change XXX below and not anything else
parameter_bounding_box=((-20, 20), (0.1, 5.0)) # specify the constraints for each parameter - some guess work...
initial_arguments = np.array([mu_init, sigma_init]) # point in 2D to initialise the minimize algorithm
result_problem2_opt = optimize.minimize(negLogLklOfIIDNormalSamples, initial_arguments, bounds=parameter_bounding_box, method="L-BFGS-B") 
# call the minimize method above finally! you need to play a bit to get initial conditions and bounding box ok
result_problem2_opt


  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 58.63138728236707
        x: [ 9.269e+00  1.708e+00]
      nit: 0
      jac: [ 0.000e+00  0.000e+00]
     nfev: 3
     njev: 1
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

---
## Assignment 2, PROBLEM 3
Maximum Points = 4



Derive the maximum likelihood estimate for $n$ IID samples from a random variable with the following probability density function:
$$
f(x; \lambda) = \frac{1}{24} \lambda^5 x^4 \exp(-\lambda x), \qquad \text{ where, } \lambda>0, x > 0
$$

You can solve the MLe by hand (using pencil paper or using key-strokes). Present your solution as the return value of a function called `def MLeForAssignment2Problem3(x)`, where `x` is a list of $n$ input data points.

In [72]:

# do not change the name of the function, just replace XXX with the appropriate expressions for the MLe
def MLeForAssignment2Problem3(x):
    '''This function returns the MLE of λ for f(x;λ) = (1/24) λ^5 x^4 exp(-λ x), λ>0, x>0
    where x is a list of n positive input data points'''
    
    x = [xi for xi in x if xi > 0]   
    n = len(x)
    if n == 0:
        raise ValueError("x must contain positive samples.")
    return 5.0 * n / sum(x)  

---
## Assignment 2, PROBLEM 4
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in ITDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [73]:

def problem4_LCG(size=None, seed = 0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    
    Parameters
    -------------
    size : an integer denoting how many samples should be produced
    seed : the starting point of the LCG, i.e. u0 in the notes.
    
    Returns
    -------------
    out : a list of the pseudo random numbers
    """
    
    a = 1664525
    b = 1013904223
    M = 2**32
    seed = seed
    output = []

    for _ in range(size):
        seed = ((a * seed) + b) % M   
        output.append(seed / M)  

    return output



In [74]:

def problem4_uniform(generator=None, period = 1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size,seed) and produces the same result as problem1_LCG, i.e. pseudo random numbers in the range {0,1,...,period-1}
    period : the period of the generator
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """
    
    ps_rand_samples = generator(size=size, seed=seed)

    unif_samples = [x / period for x in ps_rand_samples]
    
    return unif_samples

In [75]:

def problem4_accept_reject(uniformGenerator=None, n_iterations=None, seed=0):
    """
    Takes a generator that produces uniform pseudo random [0,1] numbers 
    and produces samples from (pi/2)*abs(sin(x*2*pi)) using an Accept-Reject
    sampler with the uniform distribution as the proposal distribution.
    Runs n_iterations
    
    Parameters
    -------------
    generator : a function of the type generator(size,seed) that produces uniform pseudo random
    numbers from [0,1]
    seed : the seed to be used in the generator provided
    n_iterations : an integer denoting how many attempts should be made in the accept-reject sampler
    
    Returns
    --------------
    out : a list of the pseudo random numbers with the specified distribution
    """
    
   
    unif_rn= np.array(uniformGenerator(size=2 * n_iterations, seed=seed))
    
    samples = []
    for i in range(0, 2 * n_iterations, 2):
        x = unif_rn[i]        # proposed sample from g(x)
        u = unif_rn[i + 1]    # acceptance test uniform

        r = abs(np.sin(2 * np.pi * x))  # r(x) = f(x)/(M g(x))

        # accept–reject rule
        if u <= r:
            samples.append(x)

    return samples

---
#### Local Test for Assignment 2, PROBLEM 4
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [76]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

print("LCG output: %s" % problem4_LCG(size=10, seed = 1))

period = 2**32

print("Uniform sampler %s" % problem4_uniform(generator=problem4_LCG, period = period, size=10, seed=1))

uniform_sampler = lambda size,seed: problem4_uniform(generator=problem4_LCG, period = period, size=size, seed=seed)

print("Accept-Reject sampler %s" % problem4_accept_reject(uniformGenerator = uniform_sampler,n_iterations=20,seed=1))

LCG output: [0.23645552527159452, 0.3692706737201661, 0.5042420323006809, 0.7048832636792213, 0.05054362863302231, 0.3695183543022722, 0.7747629624791443, 0.556188570568338, 0.0164932357147336, 0.6392460397910327]
Uniform sampler [5.505409214449919e-11, 8.597752864476435e-11, 1.1740299693790285e-10, 1.641184239832734e-10, 1.1768105587228739e-11, 8.603519627411668e-11, 1.8038855923319802e-10, 1.2949774287835183e-10, 3.840130687396415e-12, 1.488360668977333e-10]
Accept-Reject sampler [np.float64(5.505409214449919e-11), np.float64(1.1740299693790285e-10), np.float64(1.8038855923319802e-10), np.float64(5.831270047992193e-11), np.float64(1.375307788660518e-10), np.float64(5.4732877626841037e-11), np.float64(2.0044089630265327e-10), np.float64(1.5893083106077038e-10), np.float64(5.027791637884915e-11), np.float64(1.6425061072529445e-11), np.float64(3.0224932149117145e-11), np.float64(1.3571397770775068e-10), np.float64(6.122235140723631e-11), np.float64(2.1669569318181342e-10), np.float64(7.

In [77]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

def testUniformGenerator(size,seed):
    import random
    random.seed(seed)
    
    return [random.uniform(0,1) for s in range(size)]

print("Accept-Reject sampler %s" % problem4_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

Accept-Reject sampler [np.float64(0.763774618976614), np.float64(0.651592972722763), np.float64(0.0938595867742349), np.float64(0.8357651039198697), np.float64(0.762280082457942), np.float64(0.22876222127045265), np.float64(0.9014274576114836), np.float64(0.21659939713061338), np.float64(0.23308445025757263), np.float64(0.2187810373376886), np.float64(0.28978161459048557), np.float64(0.8375779756625729), np.float64(0.6422943629324456)]
